In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Asthma/GSE205151'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Functional Immunophenotyping of Children with Critical Status Asthmaticus Identifies Differential Gene Expression Responses in Neutrophils Exposed to a Poly(I:C) Stimulus"
!Series_summary	"We determined whether we could identify clusters of children with critical asthma by functional immunophenotyping using an intracellular viral analog stimulus."
!Series_summary	"We performed a single-center, prospective, observational cohort study of 43 children ages 6 – 17 years admitted to a pediatric intensive care unit for an asthma attack between July 2019 to February 2021."
!Series_overall_design	"Neutrophils were isolated from children, stimulated overnight with LyoVec poly(I:C), and mRNA was analyzed using a targeted Nanostring immunology array. Network analysis of the differentially expressed transcripts for the paired LyoVec poly(I:C) samples was performed."
Sample Characteristics Dictionary:
{0: ['polyic_stimulation: Unstimulated', 'polyic_stimulation

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available by inspecting the background information
is_gene_available = True

# Define keys for variable availability
trait_row = 1  # assuming 'cluster' refers to asthma status here, though it's not explicit
age_row = None  # age information is not explicitly available in the provided data
gender_row = None  # gender information is also not available in the provided data

# Define conversion functions
def convert_trait(value):
    if pd.isna(value):
        return None
    if "1" in value:
        return 1
    elif "2" in value:
        return 0
    else:
        return None

def convert_age(value):
    if pd.isna(value):
        return None
    value = value.split(':')[-1].strip()
    try:
        age = int(value)
        return age
    except ValueError:
        return None

def convert_gender(value):
    if pd.isna(value):
        return None
    gender = value.split(':')[-1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE205151', './preprocessed/Asthma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Asthma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    import os
    output_dir = './preprocessed/Asthma/trait_data/'
    os.makedirs(output_dir, exist_ok=True)
    csv_path = os.path.join(output_dir, 'GSE205151.csv')
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6205808': [1], 'GSM6205809': [1], 'GSM6205810': [0], 'GSM6205811': [0], 'GSM6205812': [1], 'GSM6205813': [1], 'GSM6205814': [0], 'GSM6205815': [0], 'GSM6205816': [0], 'GSM6205817': [0], 'GSM6205818': [0], 'GSM6205819': [0], 'GSM6205820': [1], 'GSM6205821': [1], 'GSM6205822': [0], 'GSM6205823': [0], 'GSM6205824': [0], 'GSM6205825': [0], 'GSM6205826': [0], 'GSM6205827': [0], 'GSM6205828': [1], 'GSM6205829': [1], 'GSM6205830': [0], 'GSM6205831': [0], 'GSM6205832': [0], 'GSM6205833': [0], 'GSM6205834': [1], 'GSM6205835': [1], 'GSM6205836': [1], 'GSM6205837': [1], 'GSM6205838': [1], 'GSM6205839': [1], 'GSM6205840': [0], 'GSM6205841': [0], 'GSM6205842': [1], 'GSM6205843': [1], 'GSM6205844': [0], 'GSM6205845': [0], 'GSM6205846': [0], 'GSM6205847': [0], 'GSM6205848': [1], 'GSM6205849': [1], 'GSM6205850': [1], 'GSM6205851': [1], 'GSM6205852': [1], 'GSM6205853': [1], 'GSM6205854': [1], 'GSM6205855': [1], 'GSM6205856': [1], 'GSM6205857': [1], 'GSM6205858': [0], 'GSM6205859': [0], 'GSM6205860

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7',
       'ATM', 'B2M', 'B3GAT1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR'], 'ORF': ['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR'], 'GB_ACC': ['NM_000927.3', 'NM_001090.2', 'NM_005157.3', 'NM_000022.2', 'NM_001621.3'], 'GO Annotation': ['Cell Surface,Intrinsic To Membrane,Membrane Part,Membrane,Cell Fraction,Integral To Membrane,Membrane Fraction,Establishment Of Localization,Response To Chemical Stimulus,Transport,Response To Drug,Nucleotide Binding,Purine Nucleotide Binding,Adenyl Ribonucleotide Binding,Adenyl Nucleotide Binding,Purine Ribonucleotide Binding,Atp Binding', 'Biosynthetic Process,Defense Response,Cellular Biosynthetic Process,Inflammatory Response,Response To Stress,Cellular Protein Metabolic Process,Macromolecule Biosynthetic Process,Protein Metabolic Process,Response To External Stimulus,Response To Wounding,Cellular Macromolecule Metabolic Process,Translation,Translation Regulator Activity,H', 'Organelle Part,Nuclear Part,Organelle Lumen,Nuclear Lumen,Non Membrane Bound Organ

### Step 6: Gene Identifier Mapping

In [7]:
# Skip this step as `requires_gene_mapping` is False


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Asthma/gene_data/GSE205151.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Asthma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE205151', './preprocessed/Asthma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Asthma/GSE205151.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Asthma', the least common label is '1.0' with 69 occurrences. This represents 48.25% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.

